# BERT Autocomplete Model

## Preliminaries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install datasets


In [2]:
# Import Libraries
import datasets
import json
import pandas as pd
import numpy as np
import pickle
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from datasets import DatasetDict, Dataset
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments
import torch
import random

RANDOM_SEED=420

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Extract Data Instance - US Bills

Not necessary if using dataset saved on disk

In [3]:
'''courtlistener_opinions=datasets.load_dataset("pile-of-law/pile-of-law","courtlistener_opinions")
scotus_oral_arguments=datasets.load_dataset("pile-of-law/pile-of-law","scotus_oral_arguments")
atticus_contracts=datasets.load_dataset("pile-of-law/pile-of-law","atticus_contracts")
us_bills=datasets.load_dataset("pile-of-law/pile-of-law","us_bills")
tos=datasets.load_dataset("pile-of-law/pile-of-law","tos")
federal_register=datasets.load_dataset("pile-of-law/pile-of-law","federal_register")
cfr=datasets.load_dataset("pile-of-law/pile-of-law","cfr")
uscode=datasets.load_dataset("pile-of-law/pile-of-law","uscode")
olc_memos=datasets.load_dataset("pile-of-law/pile-of-law","olc_memos")
echr=datasets.load_dataset("pile-of-law/pile-of-law","echr")'''
us_bills=datasets.load_dataset("pile-of-law/pile-of-law","us_bills")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pile-of-law.py:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/25.6k [00:00<?, ?B/s]

The repository for pile-of-law/pile-of-law contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/pile-of-law/pile-of-law.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Loading Dataset Infos from /root/.cache/huggingface/modules/datasets_modules/datasets/pile-of-law--pile-of-law/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60
INFO:datasets.info:Loading Dataset Infos from /root/.cache/huggingface/modules/datasets_modules/datasets/pile-of-law--pile-of-law/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60
Generating dataset pile-of-law (/root/.cache/huggingface/datasets/pile-of-law___pile-of-law/us_bills/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60)
INFO:datasets.builder:Generating dataset pile-of-law (/root/.cache/huggingface/datasets/pile-of-law___pile-of-law/us_bills/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60)
INFO:datasets.builder:Downloading and preparing dataset pile-of-law/us_bills to /root/.cache/huggingface/datasets/pile-of-law___pile-of-law/us_bills/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60...


...

INFO:datasets.download.download_manager:Downloading took 0.0 min
Checksum Computation took 0.0 min
INFO:datasets.download.download_manager:Checksum Computation took 0.0 min
Generating train split
INFO:datasets.builder:Generating train split


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split
INFO:datasets.builder:Generating validation split


Generating validation split: 0 examples [00:00, ? examples/s]

Unable to verify splits sizes.
INFO:datasets.utils.info_utils:Unable to verify splits sizes.
Dataset pile-of-law downloaded and prepared to /root/.cache/huggingface/datasets/pile-of-law___pile-of-law/us_bills/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60. Subsequent calls will reuse this data.
INFO:datasets.builder:Dataset pile-of-law downloaded and prepared to /root/.cache/huggingface/datasets/pile-of-law___pile-of-law/us_bills/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60. Subsequent calls will reuse this data.


Structure of `us_bills`

In [7]:
us_bills

DatasetDict({
    train: Dataset({
        features: ['text', 'created_timestamp', 'downloaded_timestamp', 'url'],
        num_rows: 84362
    })
    validation: Dataset({
        features: ['text', 'created_timestamp', 'downloaded_timestamp', 'url'],
        num_rows: 28121
    })
})

In [14]:
path='/content/drive/MyDrive/CS7650/Project/Dataset/us_bills'
os.makedirs(path,exist_ok=True)
us_bills.save_to_disk(path)

Saving the dataset (0/3 shards):   0%|          | 0/84362 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/28121 [00:00<?, ? examples/s]

### Load Data Instance from DIsk

In [8]:
path='/content/drive/MyDrive/CS7650/Project/Dataset/us_bills'
us_bills=datasets.load_from_disk(path)

In [6]:
us_bills['train'][0]['text']

'\n\t\n\n113 S2875 IS: National Guard Investigations Transparency and Improvement Act of 2014\nU.S. Senate\n2014-09-18\ntext/xml\nEN\nPursuant to Title 17 Section 105 of the United States Code, this file is not subject to copyright protection and is in the public domain.\n\n\n\n\t\tII\n\t\t113th CONGRESS2d Session\n\t\tS. 2875\n\t\tIN THE SENATE OF THE UNITED STATES\n\t\t\n\t\t\tSeptember 18, 2014\n\t\t\tMr. Begich introduced the following bill; which was read twice and referred to the Committee on Armed Services\n\t\t\n\t\tA BILL\n\t\tTo codify in law the establishment and duties of the Office of Complex Administrative\n\t\t\t Investigations in the National Guard Bureau, and for other purposes.\n\t\n\t1.Short titleThis Act may be cited as the National Guard Investigations Transparency and Improvement Act of 2014.2.Codification in law of establishment and duties of the Office of Complex Administrative\n\t\t\t Investigations in the National Guard Bureau(a)In generalThere is in the Offic

In [7]:
us_bills['validation'][0]['text']

' \n\n\n114 HR 2710 IH: Lawful Purpose and Self Defense Act\nU.S. House of Representatives\n2015-06-10\ntext/xml\nEN\nPursuant to Title 17 Section 105 of the United States Code, this file is not subject to copyright protection and is in the public domain.\n\n\n\nI \n114th CONGRESS\n1st Session\nH. R. 2710 \nIN THE HOUSE OF REPRESENTATIVES \n \nJune 10, 2015 \nMr. Bishop of Utah (for himself, Mr. Latta, Mr. Babin, Mr. Pearce, Mr. Kelly of Pennsylvania, Mr. Lamborn, Mr. Westerman, Mr. LaMalfa, Mr. Young of Alaska, Mr. Franks of Arizona, Mr. Pompeo, Mr. Smith of Texas, Mr. Cook, Mr. Hunter, Mr. Miller of Florida, Mr. Stewart, Mr. Farenthold, Mr. Chabot, Mr. Jones, Mr. Russell, Mr. Newhouse, Mr. Valadao, Mr. Cramer, Mr. Zinke, Mr. Nunes, Mr. Sessions, Mr. Hurd of Texas, Mr. Huelskamp, Mr. Olson, Mr. Gosar, Mr. Crawford, Mr. Fleming, Mr. Harper, Mr. Hudson, Mr. Crenshaw, Mr. Emmer of Minnesota, Mrs. Blackburn, Mr. Boustany, Mr. Smith of Nebraska, Mr. Graves of Georgia, Mr. McClintock, Mr. J

## NLTK Preprocessing

Run only if preprocessing is necessary, else use preprocessed data

In [14]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def nltk_preprocess(txt):
  tokens = nltk.word_tokenize(txt.lower())
  tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalnum() and token.lower() not in stop_words]
  return ' '.join(tokens)

def nltk_batch_preprocess(txts):
  preprocessed_texts=[]
  for txt in txts:
    preprocessed_texts.append(nltk_preprocess(txt))
  return preprocessed_texts

In [8]:
preprocessed_us_bills=us_bills.copy()

In [16]:
##### Takes 30min #####

for split in ['train','validation']:
  preprocessed_us_bills[split]=preprocessed_us_bills[split].map((lambda x: {'text': nltk_batch_preprocess(x['text'])}),batched=True,num_proc=4)

Map (num_proc=4):   0%|          | 0/84362 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/28121 [00:00<?, ? examples/s]

### Preprocessed Data

In [29]:
preprocessed_us_bills['train'][0]['text']

'113 s2875 national guard investigation transparency improvement act 2014 senate en pursuant title 17 section 105 united state code file subject copyright protection public domain ii 113th congress2d session 2875 senate united state september 18 2014 begich introduced following bill read twice referred committee armed service bill codify law establishment duty office complex administrative investigation national guard bureau purpose titlethis act may cited national guard investigation transparency improvement act law establishment duty office complex administrative investigation national guard bureau generalthere office chief national guard bureau office complex administrative investigation section referred office b direction supervisionthe office shall direction supervision chief national guard bureau c duty 1 generalthe duty office shall undertake complex administrative investigation matter relating member national guard state status including investigation sexual assault involving m

In [21]:
preprocessed_us_bills['validation'][0]['text']

'114 hr 2710 ih lawful purpose self defense act house representative en pursuant title 17 section 105 united state code file subject copyright protection public domain 114th congress 1st session 2710 house representative june 10 2015 bishop utah latta babin pearce kelly pennsylvania lamborn westerman lamalfa young alaska frank arizona pompeo smith texas cook hunter miller florida stewart farenthold chabot jones russell newhouse valadao cramer zinke nunes session hurd texas huelskamp olson gosar crawford fleming harper hudson crenshaw emmer minnesota blackburn boustany smith nebraska graf georgia mcclintock jody hice georgia amodei womack buck love salmon cuellar rothfus chaffetz duncan south carolina introduced following bill referred committee judiciary addition committee way mean period subsequently determined speaker case consideration provision fall within jurisdiction committee concerned bill revise various law interfere right people obtain use firearm lawful purpose titlethis act

In [31]:
preprocessed_us_bills

{'train': Dataset({
     features: ['text', 'created_timestamp', 'downloaded_timestamp', 'url'],
     num_rows: 84362
 }),
 'validation': Dataset({
     features: ['text', 'created_timestamp', 'downloaded_timestamp', 'url'],
     num_rows: 28121
 })}

In [34]:
train=preprocessed_us_bills['train']
validation=preprocessed_us_bills['validation']

In [7]:
path='/content/drive/MyDrive/CS7650/Project/Dataset/preprocessed_us_bills'

In [36]:
os.makedirs(path,exist_ok=True)
preprocessed_dataset=DatasetDict({'train':train,'validation':validation})
preprocessed_dataset.save_to_disk(path)

Saving the dataset (0/2 shards):   0%|          | 0/84362 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/28121 [00:00<?, ? examples/s]

## BERT (Pre-trained)

In [20]:
path='/content/drive/MyDrive/CS7650/Project/Dataset/preprocessed_us_bills'
preprocessed_us_bills=datasets.load_from_disk(path)

In [21]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [22]:
# Masking Function
def mask_dataset(data):
  masked_sentences=[]
  labels=[]
  for text in data['text']:
    tokens=tokenizer.tokenize(text)
    mask_index=random.randint(0,len(tokens)-1)

    masked_tokens=tokens[:mask_index]+['[MASK]']+tokens[mask_index+1:]
    masked_sentence=tokenizer.convert_tokens_to_string(masked_tokens)

    masked_sentences.append(' '.join(masked_sentence))
    labels.append(tokens[mask_index])
  return masked_sentences,labels

In [51]:
def mask_tokens(data,mask_prob=0.2,max_length=120):
    masked_sentences = []
    labels = []
    attention_masks = []

    for text in data['text']:
        # Tokenize the text
        tokens = tokenizer.tokenize(text)
        token_ids = tokenizer.convert_tokens_to_ids(tokens)[:max_length]

        attention_mask = [1] * len(token_ids)

        padded_token_ids = token_ids[:max_length] + [tokenizer.pad_token_id] * (max_length - len(token_ids))
        attention_mask += [0] * (max_length - len(attention_mask))

        # Copy token_ids to labels, but mask all non-masked tokens with -100
        label = [-100] * max_length

        # Mask random tokens (e.g., 15% of the tokens)
        num_to_mask = max(1, int(len(token_ids) * mask_prob))
        mask_indices = random.sample(range(len(token_ids)), num_to_mask)

        for mask_index in mask_indices:
            label[mask_index] = padded_token_ids[mask_index] # Keep the original token for the masked index
            padded_token_ids[mask_index] = tokenizer.convert_tokens_to_ids('[MASK]')  # Mask the token

        # Append to the lists
        masked_sentences.append(token_ids)
        labels.append(label)
        attention_masks.append(attention_mask)

    return masked_sentences, labels, attention_masks

In [33]:
# Tokenizing Function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [52]:
# Only Use First 1000 bills for training
train_data=preprocessed_us_bills['train'][0:1000]
masked_sentences,labels,attention_masks=mask_tokens(train_data)

In [53]:
# Generate Masked Dataset
train_data_masked=Dataset.from_dict({'input_ids':masked_sentences,'labels':labels,'attention_masks':attention_masks})
train_data_masked

Dataset({
    features: ['input_ids', 'labels', 'attention_masks'],
    num_rows: 1000
})

In [54]:
# Save to disk as a checkpoint
train_data_masked.save_to_disk('/content/drive/MyDrive/CS7650/Project/Dataset/train_data_masked')

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [55]:
train_data_masked=datasets.load_from_disk('/content/drive/MyDrive/CS7650/Project/Dataset/train_data_masked')
train_data_masked

Dataset({
    features: ['input_ids', 'labels', 'attention_masks'],
    num_rows: 1000
})

In [58]:
# Load pretrained BERT
model=BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [59]:
# Training args
training_args = TrainingArguments(
    output_dir='./results',           # output directory
    evaluation_strategy='epoch',      # evaluate every epoch
    learning_rate=2e-5,               # learning rate
    per_device_train_batch_size=8,    # batch size per device during training
    num_train_epochs=3,               # number of training epochs
    weight_decay=0.01,                # strength of weight decay
    logging_dir='./logs',             # directory for storing logs
    logging_steps=10,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [60]:
# Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data_masked,

)


In [61]:
# Start training
trainer.train()

ValueError: expected sequence of length 120 at dim 1 (got 101)

In [42]:
# Save Trained Model
trainer.save_model('/content/drive/MyDrive/CS7650/Project/BERT_trained_autocomplete_model')
